In [11]:
import pandas as pd
chunk_size = 10000
batch_no = 1
file_name = "C://Users//Harish//Desktop//malware_train.csv"
for chunk in pd.read_csv(file_name,chunksize=chunk_size):
    chunk.to_csv('malware'+str(batch_no)+'.csv',index=False)
    batch_no+=1

C:\Users\Harish\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: 

In [ ]:
import warnings
import numpy as np
import pandas as pd
import pickle
import dill
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
# Donot modify the method signatures and return dtypes
# you can include additional functions of your choice

class MalwareDetectionModel:

    def __init__(self):
        self.dtypes = {
            'MachineIdentifier':                                    'category',
            'ProductName':                                          'category',
            'EngineVersion':                                        'category',
            'AppVersion':                                           'category',
            'AvSigVersion':                                         'category',
            'IsBeta':                                               'int8',
            'RtpStateBitfield':                                     'float16',
            'IsSxsPassiveMode':                                     'int8',
            'DefaultBrowsersIdentifier':                            'float32',  # was 'float16'
            'AVProductStatesIdentifier':                            'float32',
            'AVProductsInstalled':                                  'float16',
            'AVProductsEnabled':                                    'float16',
            'HasTpm':                                               'int8',
            'CountryIdentifier':                                    'int16',
            'CityIdentifier':                                       'float32',
            'OrganizationIdentifier':                               'float16',
            'GeoNameIdentifier':                                    'float16',
            'LocaleEnglishNameIdentifier':                          'int16',  # was 'int8'
            'Platform':                                             'category',
            'Processor':                                            'category',
            'OsVer':                                                'category',
            'OsBuild':                                              'int16',
            'OsSuite':                                              'int16',
            'OsPlatformSubRelease':                                 'category',
            'OsBuildLab':                                           'category',
            'SkuEdition':                                           'category',
            'IsProtected':                                          'float16',
            'AutoSampleOptIn':                                      'int8',
            'PuaMode':                                              'category',
            'SMode':                                                'float16',
            'IeVerIdentifier':                                      'float16',
            'SmartScreen':                                          'category',
            'Firewall':                                             'float16',
            'UacLuaenable':                                         'float64', # was 'float32'
            'Census_MDC2FormFactor':                                'category',
            'Census_DeviceFamily':                                  'category',
            'Census_OEMNameIdentifier':                             'float32', # was 'float16'
            'Census_OEMModelIdentifier':                            'float32',
            'Census_ProcessorCoreCount':                            'float16',
            'Census_ProcessorManufacturerIdentifier':               'float16',
            'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
            'Census_ProcessorClass':                                'category',
            'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
            'Census_PrimaryDiskTypeName':                           'category',
            'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
            'Census_HasOpticalDiskDrive':                           'int8',
            'Census_TotalPhysicalRAM':                              'float32',
            'Census_ChassisTypeName':                               'category',
            'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
            'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
            'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
            'Census_PowerPlatformRoleName':                         'category',
            'Census_InternalBatteryType':                           'category',
            'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
            'Census_OSVersion':                                     'category',
            'Census_OSArchitecture':                                'category',
            'Census_OSBranch':                                      'category',
            'Census_OSBuildNumber':                                 'int16',
            'Census_OSBuildRevision':                               'int32',
            'Census_OSEdition':                                     'category',
            'Census_OSSkuName':                                     'category',
            'Census_OSInstallTypeName':                             'category',
            'Census_OSInstallLanguageIdentifier':                   'float16',
            'Census_OSUILocaleIdentifier':                          'int16',
            'Census_OSWUAutoUpdateOptionsName':                     'category',
            'Census_IsPortableOperatingSystem':                     'int8',
            'Census_GenuineStateName':                              'category',
            'Census_ActivationChannel':                             'category',
            'Census_IsFlightingInternal':                           'float16',
            'Census_IsFlightsDisabled':                             'float16',
            'Census_FlightRing':                                    'category',
            'Census_ThresholdOptIn':                                'float16',
            'Census_FirmwareManufacturerIdentifier':                'float16',
            'Census_FirmwareVersionIdentifier':                     'float32',
            'Census_IsSecureBootEnabled':                           'int8',
            'Census_IsWIMBootEnabled':                              'float16',
            'Census_IsVirtualDevice':                               'float16',
            'Census_IsTouchEnabled':                                'int8',
            'Census_IsPenCapable':                                  'int8',
            'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
            'Wdft_IsGamer':                                         'float16',
            'Wdft_RegionIdentifier':                                'float16',
            'HasDetections':                                        'int8',
        }
        self.file_name = "malware10.csv"
        self.clf = DecisionTreeClassifier()
        
    def clean(self, test_data):
        test_data = test_data.drop(columns=self.null_cols)
        test_data = test_data.drop(columns=self.skew_cols)

        # clean up NaN values
        for column in self.num_cols:
            test_data[column].fillna(test_data[column].median(), inplace=True)
        for column in self.cat_cols:
            test_data[column].fillna(test_data[column].value_counts().index[0],inplace=True)

        # encode string values
        test_data = pd.get_dummies(test_data,columns=self.cat_cols,drop_first=True)

        return test_data

    def _clean(self, df):
        
        #drop columns with most missing values
        total = df.isnull().sum().sort_values(ascending = False)
        percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
        null_cols  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        self.null_cols = null_cols[(null_cols['Percent']>5)].index.tolist()
        df = df.drop(columns=self.null_cols)
        
        
        #drop columns with large percentage of values in one category
        skew_percent = pd.DataFrame([{'column':c,'skewness': df[c].value_counts(normalize=True).values[0] * 100} for c in df.columns]).sort_values('skewness', ascending=False)
        skew_percent.set_index('column',inplace=True)
        self.skew_cols = skew_percent[(skew_percent['skewness']>99.0)].index.tolist()
        df = df.drop(columns=self.skew_cols)
        
        
        # set index to be the machine IDs 
#         df.set_index('MachineIdentifier', inplace=True)
        df.drop(columns=['MachineIdentifier'],inplace=True)


        # clean up NaN values
        self.num_cols = df.select_dtypes(['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns.tolist()
        self.cat_cols = df.select_dtypes(['category']).columns.tolist()
        for column in self.num_cols:
            df[column].fillna(df[column].median(), inplace=True)
        for column in self.cat_cols:
            df[column].fillna(df[column].value_counts().index[0],inplace=True)

        # One Hot Encoding of categorical variables
        df = pd.get_dummies(df,columns=self.cat_cols,drop_first=True)
        

        return df


    # trains a model, you can access the malware_train.csv file for training your model
    def train(self):
        # read in csv into dataframe
        df = pd.read_csv(self.file_name, dtype=self.dtypes)
        df = self._clean(df)
        col_list = df.columns.tolist()
        index = col_list.index('HasDetections')


        # prediction column
        y_train = df.iloc[:,index].values
        print((y_train))
        # feature columns
        x_train = df.drop('HasDetections',1).values
       
        #normalize the data
        x_train = preprocessing.normalize(x_train)
        self.clf = GradientBoostingClassifier()
        self.clf.fit(x_train, y_train)
        scores = cross_val_score(self.clf, x_train, y_train, cv=10, n_jobs=-1)
        print(scores.mean())
        print(scores.std())


# #         training
#         self.clf = Sequential()
#         # Adding the input layer and the first hidden layer
#         self.clf.add(Dense(output_dim = 37, init = 'uniform', activation = 'relu', input_dim = len(x_train)))
#         # Adding the second hidden layer
#         self.clf.add(Dense(output_dim = 37, init = 'uniform', activation = 'relu'))
#         # Adding the output layer
#         self.clf.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
#         # Compiling Neural Network
#         self.clf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#         # Fitting our model 
#         self.clf.fit(x_train, y_train, batch_size = 100, nb_epoch = 100)
#         scores = cross_val_score(self.clf, x_train, y_train, cv=10, n_jobs=-1)
#         print(scores.mean())


    # predicts the probablity of a machine affected by malware. The input test_data is a pandas dataframe.
    # This function must return the predicted probabilities for the test machines
    def predict_probablities(self, test_data):
        print('Predict malware')
        test_data = self.clean(test_data)
        test_data = preprocessing.normalize(test_data).to_frame()
        predictions = self.clf.predict(test_data)
        return predictions


# calls the train function and saves the model_file
# # donot modify the contents
if __name__ == "__main__":
    detectionModel = MalwareDetectionModel()
    detectionModel.train()
    filename = 'malwaremodel.pkl'
    f = open(filename, 'wb')
    dill.dump(detectionModel, f)
    f.close()
    
    
# dm = MalwareDetectionModel()

[1 1 0 ... 1 1 0]
